In [1]:
import yfinance as yf
from datetime import datetime, timedelta
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from tqdm import tqdm
import pandas as pd
from statsmodels.tools.sm_exceptions import ValueWarning, HessianInversionWarning, ConvergenceWarning
import warnings
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from pandasgui import show
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split

WARN: Could not locate executable g77
WARN: Could not locate executable f77
WARN: Could not locate executable ifort
WARN: Could not locate executable ifl
WARN: Could not locate executable f90
WARN: Could not locate executable DF
WARN: Could not locate executable efl


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [98]:
dfInfoEmpresas = pd.read_csv('informationCompanies.csv')

In [93]:
dfInfoEmpresas[['marketCap','symbol']]

,marketCap,symbol
0,5.150962e+10,MMM
1,9.626784e+09,AOS
2,1.756687e+11,ABT
3,2.425370e+11,ABBV
4,1.935059e+11,ACN
...,...,...
493,3.558785e+10,YUM
494,1.351511e+10,ZBRA
495,2.640098e+10,ZBH
496,4.040935e+09,ZION


In [99]:
dfInfoEmpresas = dfInfoEmpresas[dfInfoEmpresas['fullTimeEmployees'].notna()]
dfInfoEmpresas.reset_index()
dfInfoEmpresas =dfInfoEmpresas.drop(['lastSplitFactor', 'lastSplitDate',
                     'fiveYearAvgDividendYield','dividendRate',
                     'dividendYield','industrySymbol',
                     'address2','trailingPegRatio',
                     'lastDividendDate', 'maxAge', 'previousClose', 'open', 
                    'dayLow', 'dayHigh', 'regularMarketPreviousClose',
                    'regularMarketOpen',  'regularMarketDayLow',
                    'regularMarketDayHigh', 'trailingPE', 
                    'governanceEpochDate', 'compensationAsOfEpochDate',
                    'gmtOffSetMilliseconds'], axis=1)
dfInfoEmpresas = dfInfoEmpresas.reset_index()


In [100]:
nombres = dfInfoEmpresas[['symbol','shortName']]

In [101]:
nombres

,symbol,shortName
0,MMM,3M Company
1,AOS,A.O. Smith Corporation
2,ABT,Abbott Laboratories
3,ABBV,AbbVie Inc.
4,ACN,Accenture plc
...,...,...
493,YUM,"Yum! Brands, Inc."
494,ZBRA,Zebra Technologies Corporation
495,ZBH,"Zimmer Biomet Holdings, Inc."
496,ZION,Zions Bancorporation N.A.


In [102]:
dfInfoEmpresas = dfInfoEmpresas.drop(['symbol','shortName'], axis=1)
dfInfoEmpresas = dfInfoEmpresas.reset_index()


In [103]:
dfInfoEmpresas

,level_0,index,Unnamed: 0,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,priceHint,...,returnOnAssets,returnOnEquity,grossProfits,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins
0,0,0,0,92000.0,10.0,5.0,8.0,4.0,7.0,2.0,...,0.06742,0.36034,1.500000e+10,3.498250e+09,5.855000e+09,-0.221,-0.090,0.42547,0.20460,0.14962
1,1,1,1,12000.0,9.0,10.0,8.0,10.0,10.0,2.0,...,0.12177,0.13492,1.329600e+09,3.090125e+08,4.948000e+08,0.105,-0.012,0.36394,0.19649,0.17624
2,2,2,2,115000.0,7.0,6.0,5.0,2.0,4.0,2.0,...,0.06318,0.15932,2.457900e+10,6.409875e+09,8.659000e+09,-0.453,-0.181,0.55639,0.25831,0.18000
3,3,3,3,50000.0,9.0,7.0,7.0,9.0,9.0,2.0,...,0.09443,0.51275,4.152700e+10,2.160288e+10,2.422800e+10,-0.949,-0.097,0.70959,0.52031,0.36979
4,4,4,4,738000.0,1.0,3.0,3.0,4.0,1.0,2.0,...,0.13110,0.30949,1.970154e+10,8.329611e+09,9.680274e+09,-0.059,0.051,0.32120,0.16945,0.15288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,493,498,498,36000.0,1.0,2.0,10.0,7.0,8.0,2.0,...,0.23843,NaN,3.307000e+09,7.075000e+08,1.523000e+09,-0.228,0.063,0.48501,0.33775,0.31787
494,494,499,499,10500.0,1.0,9.0,8.0,9.0,8.0,2.0,...,0.08776,0.14076,2.624000e+09,7.526250e+08,3.580000e+08,-0.243,-0.019,0.46124,0.20003,0.16597
495,495,500,500,18000.0,2.0,8.0,10.0,4.0,8.0,2.0,...,0.04118,0.03728,4.920400e+09,1.391900e+09,1.355200e+09,15.353,0.101,0.71576,0.32870,0.19864
496,496,501,501,10064.0,7.0,7.0,4.0,5.0,5.0,2.0,...,0.01010,0.15822,3.030000e+09,NaN,1.403000e+09,0.047,0.104,0.00000,0.00000,0.39581


In [8]:
show(dfInfoEmpresas)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [104]:
print((((dfInfoEmpresas).isna()).sum()).to_string())

level_0                          0
index                            0
Unnamed: 0                       0
fullTimeEmployees                0
auditRisk                        4
boardRisk                        4
compensationRisk                 4
shareHolderRightsRisk            4
overallRisk                      4
priceHint                        0
exDividendDate                  72
payoutRatio                     24
beta                             4
forwardPE                        1
volume                           0
regularMarketVolume              0
averageVolume                    0
averageVolume10days              0
averageDailyVolume10Day          0
bid                              0
ask                              0
bidSize                          0
askSize                          0
marketCap                        0
fiftyTwoWeekLow                  0
fiftyTwoWeekHigh                 0
priceToSalesTrailing12Months     2
fiftyDayAverage                  0
twoHundredDayAverage

In [105]:
imputer = KNNImputer(n_neighbors=100)

In [106]:
dfImputado = imputer.fit_transform(dfInfoEmpresas)

In [107]:
nombreColumnas = list(dfInfoEmpresas.columns)

In [109]:
dfImputado = pd.DataFrame(dfImputado, columns = nombreColumnas)

In [117]:
column = nombres['symbol']
dfImputado.insert(0, 'symbol', column)

In [119]:
column = nombres['shortName']
dfImputado.insert(0, 'shortName', column)

In [123]:
dfImputado = dfImputado.drop(['Unnamed: 0','level_0', 'index'], axis=1)

In [125]:
dfImputado.to_csv('infoImputadaVer2.csv') 